In [110]:
import ghmm
from collections import OrderedDict
import cPickle as pickle
import numpy as np
from itertools import product as iterproduct, chain
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
import math
import random
from nbwrapper import getargs
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/alignment_validation.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/alignment_lib.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/analysis_lib.ipynb"

In [111]:
args = getargs()

In [112]:
NMERS = 6

In [113]:
# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.called",
#     "ref": "/home/ibis/gregor.sturm/nanopore/NanoporeData/PublicData/LomanLab_MAP-006/ecoli_mg1655.fa",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1.model.pickle",
#     "corr_model": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/context_prediction/model-test.pickle",
#     "ncores": 62,
#     "nmers": NMERS,
#     "multivariate": False
# }
args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/wouter_lambda006_100.events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.called",
    "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1.model.pickle",
    "corr_model": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/context_prediction/model-test2.pickle",
    "ncores": 62,
    "nmers": NMERS,
    "multivariate": False
}


In [114]:
NMERS = int(args["nmers"])
NSTATES = 4**NMERS
MULTIVARIATE = bool(int(args["multivariate"]))
args["ncores"] = int(args["ncores"])

In [115]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = HMM_PARAMS["/opt/chimaera/model/r7.3_e6_70bps_6mer/template_median68pA.model"]
ALL_KMERS = ["".join(x) for x in iterproduct("ACGT", repeat=NMERS)]
assert HMM_PARAMS["kmer"].tolist() == ALL_KMERS

# Train Model 

In [116]:
def mk_transmat1(nmers):
    """make a transition matrix assuming move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [117]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [118]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (2/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (47/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [119]:
mk_transmat = mk_transmat2

In [120]:
def mk_model_simple(): 
    """ simple model, only taking the means into account. """
    A = mk_transmat(NMERS)
    B = HMM_PARAMS[["level_mean", "level_stdv"]].values.tolist() #mu, std of each state
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state
    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [121]:
F = ghmm.Float()  # emission domain of this model
def mk_model():
    if MULTIVARIATE: 
        return mk_model_multivariate()
    else: 
        return mk_model_simple()

In [122]:
model = mk_model()
s = str(model)
print(s)

GaussianEmissionHMM(N=4096)
  state 0 (initial=0.00, mu=62.78, sigma=0.84)
    Transitions: ->0 (0.00), ->1 (0.00), ->2 (0.00), ->3 (0.00), ->4 (0.00), ->5 (0.00), ->6 (0.00), ->7 (0.00), ->8 (0.00), ->9 (0.00), ->10 (0.00), ->11 (0.00), ->12 (0.00), ->13 (0.00), ->14 (0.00), ->15 (0.00)
  state 1 (initial=0.00, mu=58.02, sigma=0.66)
    Transitions: ->1 (0.02), ->4 (0.23), ->5 (0.23), ->6 (0.23), ->7 (0.23), ->16 (0.00), ->17 (0.00), ->18 (0.00), ->19 (0.00), ->20 (0.00), ->21 (0.00), ->22 (0.00), ->23 (0.00), ->24 (0.00), ->25 (0.00), ->26 (0.00), ->27 (0.00), ->28 (0.00), ->29 (0.00), ->30 (0.00), ->31 (0.00)

  ...

  state 4094 (initial=0.00, mu=45.36, sigma=0.64)
    Transitions: ->4064 (0.00), ->4065 (0.00), ->4066 (0.00), ->4067 (0.00), ->4068 (0.00), ->4069 (0.00), ->4070 (0.00), ->4071 (0.00), ->4072 (0.00), ->4073 (0.00), ->4074 (0.00), ->4075 (0.00), ->4076 (0.00), ->4077 (0.00), ->4078 (0.00), ->4079 (0.00), ->4088 (0.23), ->4089 (0.23), ->4090 (0.23), ->4091 (0.23), ->409

In [123]:
def result_to_seq(result):
    states = result[0]
    kmers = [ALL_KMERS[x] for x in states]
    seq = [kmer[0] for kmer in kmers] + [kmers[-1][1:]]
    return "".join(seq)

In [124]:
def predict(events):
    """mixed is a set of tuples (event_mean, event_stdv)"""
    emissions = [x[0] for x in events]
    seq = ghmm.EmissionSequence(F, emissions)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [125]:
s = model.sampleSingle(10)
s = [x for x in s]
seq = zip([s[i] for i in range(0, len(s), 2)], [s[i] for i in range(1, len(s), 2)])

In [126]:
predict(seq)

'TAGTAGAATT'

In [127]:
model

## multistep-prediction

In [128]:
from sklearn import ensemble
import joblib
import mltools

In [129]:
corr_model = joblib.load(args["corr_model"])

In [130]:
OFFSET = 20

In [131]:
def normalize_col(features, col): 
    col_data = [f[col] for f in features]
    col_data = mltools.normalize(col_data)
    for i, f in enumerate(features): 
        f[col] = col_data[i]
    return features

In [132]:
def correct_events(events, seq): 
    corr_range = (OFFSET, len(seq)-OFFSET-NMERS)
#     print (">> started feature generation")
    features = []
    for i in range(*corr_range):
        event = events[i]
        feat = []
        feat.append(event[0])
        feat.append(event[1])
        feat.extend(mltools.seq2binvec(seq[i-OFFSET:i+NMERS+OFFSET]))
        features.append(feat)

    features = normalize_col(features, 0)
    features = normalize_col(features, 1)
    
    #print([x for x in enumerate(features[0])])
        
#     print (">> started correction prediction")
    correction = corr_model.predict(features)
    for i, j  in enumerate(range(*corr_range)):
        mean, stdv = events[j]
        tmp_mean = mean - .1 * correction[i]
        ratio = tmp_mean/mean
        tmp_stdv = ratio * stdv
        events[j] = tmp_mean, tmp_stdv
#     print (">> correction applied")
    return events

In [133]:
def predict_iterative(events, n_steps=3): 
    for _ in range(n_steps):
#         print (">> predict {0}".format(_))
        seq = predict(events)
        if _ < n_steps-1: 
#             print (">> correct {0}".format(_))
#             print(seq)
#             print(means)
            events = correct_events(events, seq)
#             print(means)
    return seq

In [134]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [135]:
file_obj = correct_read(file_data[7], col="mean")
events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]

In [136]:
seq = predict(events)

In [137]:
seq[18:40]

'TTTTTCTGAGTTTCTATGTTCT'

In [138]:
events[18:30]

[(42.0114784371039, 1.4727646945616364),
 (55.9587437487013, 1.550074569098511),
 (53.15756088466786, 1.4043851028579888),
 (58.20333326574283, 1.288266778560844),
 (56.59134890130535, 1.2428741876687441),
 (59.67519172629616, 1.2614077552503005),
 (51.35883735412172, 1.8434463365497897),
 (47.01933849506197, 1.0261942757723495),
 (41.604257702418096, 1.5104981235551942),
 (52.01943797533074, 1.3689371700490918),
 (50.070775821028185, 1.2075285245719574),
 (52.7758805257471, 0.9941203401742386)]

In [139]:
corr_range = (OFFSET, len(seq)-OFFSET-NMERS)
#     print (">> started feature generation")
features = []
for i in range(*corr_range):
    event = events[i]
    feat = []
    feat.append(event[0])
    feat.append(event[1])
    feat.extend(mltools.seq2binvec(seq[i-OFFSET:i+NMERS+OFFSET]))
    features.append(feat)

features = normalize_col(features, 0)
features = normalize_col(features, 1)

#print([x for x in enumerate(features[0])])

In [140]:
[x for x in enumerate(features[0])]

[(0, 0.35172653679318944),
 (1, 0.2532404434099545),
 (2, 1),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 0),
 (10, 0),
 (11, 1),
 (12, 0),
 (13, 0),
 (14, 0),
 (15, 0),
 (16, 1),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 0),
 (21, 1),
 (22, 0),
 (23, 0),
 (24, 0),
 (25, 1),
 (26, 0),
 (27, 0),
 (28, 1),
 (29, 0),
 (30, 0),
 (31, 0),
 (32, 0),
 (33, 1),
 (34, 0),
 (35, 0),
 (36, 0),
 (37, 1),
 (38, 0),
 (39, 0),
 (40, 0),
 (41, 1),
 (42, 1),
 (43, 0),
 (44, 0),
 (45, 0),
 (46, 0),
 (47, 0),
 (48, 1),
 (49, 0),
 (50, 0),
 (51, 0),
 (52, 0),
 (53, 1),
 (54, 0),
 (55, 1),
 (56, 0),
 (57, 0),
 (58, 0),
 (59, 0),
 (60, 0),
 (61, 1),
 (62, 0),
 (63, 1),
 (64, 0),
 (65, 0),
 (66, 0),
 (67, 0),
 (68, 0),
 (69, 1),
 (70, 0),
 (71, 0),
 (72, 0),
 (73, 1),
 (74, 0),
 (75, 0),
 (76, 0),
 (77, 1),
 (78, 0),
 (79, 0),
 (80, 0),
 (81, 1),
 (82, 0),
 (83, 0),
 (84, 0),
 (85, 1),
 (86, 0),
 (87, 0),
 (88, 0),
 (89, 1),
 (90, 0),
 (91, 0),
 (92, 0),
 (93, 1),
 (94, 0),
 (95, 1),
 (96, 0),
 (97,

In [141]:
#     print (">> started correction prediction")
correction = corr_model.predict(features)

In [142]:
OFFSET

20

In [143]:
correction[:10]

array([-0.53169365,  1.38059966, -0.13873391, -0.56596379,  1.78664566,
        0.48308305,  0.22211348,  0.87392437, -0.82151468, -0.39801699])

In [144]:
events[OFFSET:OFFSET+10]

[(53.15756088466786, 1.4043851028579888),
 (58.20333326574283, 1.288266778560844),
 (56.59134890130535, 1.2428741876687441),
 (59.67519172629616, 1.2614077552503005),
 (51.35883735412172, 1.8434463365497897),
 (47.01933849506197, 1.0261942757723495),
 (41.604257702418096, 1.5104981235551942),
 (52.01943797533074, 1.3689371700490918),
 (50.070775821028185, 1.2075285245719574),
 (52.7758805257471, 0.9941203401742386)]

In [145]:
for i, j  in enumerate(range(*corr_range)):
    mean, stdv = events[j]
    tmp_mean = mean - .5 * correction[i]
    ratio = tmp_mean/mean
    tmp_stdv = ratio * stdv
    events[j] = tmp_mean, tmp_stdv
#     print (">> correction applied")

In [146]:
events[OFFSET:OFFSET+10]

[(53.423407707286991, 1.4114085875912044),
 (57.513033435219526, 1.2729877509001934),
 (56.660715855520294, 1.2443976430827741),
 (59.95817361889921, 1.2673893959220572),
 (50.465514523041506, 1.8113818899004939),
 (46.777796969839486, 1.0209226463007721),
 (41.493200961783224, 1.5064660603097206),
 (51.582475788535191, 1.3574381265628082),
 (50.481533162356662, 1.2174345265900681),
 (52.97488902126711, 0.99786899185551936)]

# Validate Model 

In [147]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks


In [148]:
assert os.path.isfile(args["events"])

In [149]:
ref = load_ref(args["ref"])

['>burn-in lambda_ref']
GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACC


In [150]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [151]:
prepare_filemap(file_data)

In [152]:
def basecall_read(file_obj):
    file_obj = correct_read(file_obj, col="mean")
    events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]
    called_seq = predict_iterative(events)
    return (file_obj["channel"], file_obj["file_id"], called_seq)

In [153]:
# """ train with baum-welch """
# for i, file_obj in enumerate(file_data): 
#     sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
#     train_read(file_obj)

In [154]:
p = Pool(args["ncores"])

In [ ]:
#basecall_read(file_data[3])

In [ ]:
print("Prediction: ")
results = []
try:
    for i, res in enumerate(p.imap_unordered(basecall_read, file_data), 1):
        results.append(res)
        sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
    p.close()
    p.join()
except KeyboardInterrupt:
    p.terminate()

### Stats

In [ ]:
types = ["metrichor", "called", "random"]
fasta_files = {t: "{0}.{1}.fa".format(args["out_basename"], t) for t in types}

In [ ]:
## metrichor fasta
with open(fasta_files["metrichor"], 'w') as f: 
    for file_obj in file_data: 
        f.write(">ch{0}_file{1}_metrichor".format(file_obj["channel"], file_obj["file_id"])+ "\n")
        f.write(file_obj["fastq"].split("\n")[1] + "\n")

In [ ]:
## called fasta/random fasta
with open(fasta_files["called"], 'w') as f: 
    with open(fasta_files["random"], 'w') as fr:
        for channel, file_id, seq in results: 
            f.write(">ch{0}_file{1}_called".format(channel, file_id)+ "\n")
            fr.write(">ch{0}_file{1}_random".format(channel, file_id)+ "\n")
            f.write(seq + "\n")
            fr.write("".join([random.choice("ACGT") for _ in range(len(seq))]))

In [ ]:
for t in types: 
    sam_file = "{0}.{1}.sam".format(args["out_basename"], t)
    graphmap(args["ref"], fasta_files[t], sam_file, args["ncores"])
    prepare_sam("{0}.{1}".format(args["out_basename"], t))

In [ ]:
def mk_stat(t):
    samfile = "{0}.{1}.sorted.bam".format(args["out_basename"], t)
    sst = samstats(samfile, ref, ncores=args["ncores"])
    return pandas.DataFrame(sst.print_summary())

In [ ]:
stats = map(mk_stat, types)
print(types)
side_by_side(*stats)

In [ ]:
# for t, df in zip(types, stats):
#     with open("{0}.stats.{1}.html".format(args["out_basename"], t), 'w') as f:
#         f.write(df.to_html())

In [ ]:
# def score_consensus(t):
#     consensus = mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], t), ref_file)
#     return(consensus)
#     consensus = consensus.split("\n")[1].to_upper()
#     score = needle(ref, consensus)
#     return (consensus, score)

In [ ]:
# p = Pool(args["ncores"])
# try:
#     consensus = p.map(score_consensus, types)
#     p.close()
# except KeyboardInterrupt:
#     p.terminate()

In [ ]:
# consensus

In [ ]:
# mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], "metrichor"), ref_file)